## Segmenting and Clustering Neirghborhoods in Toronto

### a. Creating dataframe

1. Scraping the table of wikipedia page

In [189]:
import pandas as pd
import numpy as np

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)

df = tables[0].iloc[0:]

df.columns = ['Postcode','Borough','Neighbourhood']

df.head()


Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

2. Droping rows with Borough='Not assigned'

In [190]:
df=df[df.loc[:,'Borough']!='Not assigned'].reset_index(drop=True)
df.head()

Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights

3. Changing the 'Not assigned' in Neighbourhoold cell by the Borough value

In [191]:
for i in range(df.shape[0]):
    if df.iloc[i,2]=='Not assigned':
        df.iloc[i,2]=df.iloc[i,1]
#Checking that there is no 'Not assigned' value on Neighbourhood column
(df['Neighbourhood']=='Not assigned').value_counts()

False    211
Name: Neighbourhood, dtype: int64

4. Merging Neighbourhoods separated by commas with common Postcode

In [192]:
from pandas import Series 
def f(x):
     return Series(dict(Neighbourhood = "%s" % ', '.join(x['Neighbourhood'])))
df=df.groupby(['Postcode','Borough']).apply(f)
df.head()


Neighbourhood
Postcode Borough                                            
M1B      Scarborough                          Rouge, Malvern
M1C      Scarborough  Highland Creek, Rouge Hill, Port Union
M1E      Scarborough       Guildwood, Morningside, West Hill
M1G      Scarborough                                  Woburn
M1H      Scarborough                               Cedarbrae

In [193]:
df.reset_index(inplace=True)
df.head()

Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae

In [194]:
df.shape

(103, 3)

### b. Adding latitude and longitude

In [195]:
#df['Latitude']=np.nan
#df['Longitude']=np.nan

df_latlon=pd.read_csv('http://cocl.us/Geospatial_data')
df_latlon.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [196]:
df_latlon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_latlon.head()

Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476

In [197]:
df_merged=pd.merge(df,df_latlon, on='Postcode')
df_merged.head()

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [198]:
df_toronto=df_merged

### c. Clustering neighborhoods in Toronto

#### 1. Download and Explore Dataset

In [199]:
#Make the appropiate imports
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [200]:
#We work only with the boroughs than contains the word 'Toronto'
df_toronto_filtered=df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto_filtered.head()


Postcode          Borough                   Neighbourhood   Latitude  \
0      M4E     East Toronto                     The Beaches  43.676357   
1      M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2      M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
3      M4M     East Toronto                 Studio District  43.659526   
4      M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

In [201]:
#Getting lat and lon of Toronto
#address =  'Toronto, ON'

#geolocator = Nominatim(user_agent="ny_explorer")
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude

#Geolocator is nor responding well so I hadcoded lat an lon from Toronto
latitude= 43.651070
longitude= -79.347015
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65107, -79.347015.


In [202]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_filtered['Latitude'], df_toronto_filtered['Longitude'], df_toronto_filtered['Borough'], df_toronto_filtered['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [203]:
#My Foursquare api credentials
CLIENT_ID = 'XTQAPH10RWKRAHYMCLY511LLPMUGMH1524KK3MHJOXIJ4VKY' # your Foursquare ID
CLIENT_SECRET = 'MPTM4BZH0UK20QAJIHRSZ4Z4T3IXWB42J2TOO0MH4510UGTH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XTQAPH10RWKRAHYMCLY511LLPMUGMH1524KK3MHJOXIJ4VKY
CLIENT_SECRET:MPTM4BZH0UK20QAJIHRSZ4Z4T3IXWB42J2TOO0MH4510UGTH


In [204]:
#Exploring the first neighbordhood
neighborhood_latitude = df_toronto_filtered.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_filtered.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto_filtered.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [205]:
#Now, let's get the top 20 venues that are in The Beaches within a radius of 200 meters
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION,200, 20)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc0467186bc49002c472ae5'},
 'response': {'venues': [{'id': '56afcad6498e05333bf42031',
    'name': 'Glen Stewart Ravine',
    'location': {'lat': 43.67629984029563,
     'lng': -79.2947841389563,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.67629984029563,
       'lng': -79.2947841389563}],
     'distance': 141,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d162941735',
      'name': 'Other Great Outdoors',
      'pluralName': 'Other Great Outdoors',
      'shortName': 'Other Outdoors',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/outdoors_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1572882033',
    'hasPerk': False},
   {'id': '4bd461bc77b29c74a07d9282',
    'name': 'Glen Manor Ravine',
    'location': {'address': 'Glen Manor',
     'crossStreet': 'Queen St.',
     'lat': 43.67682094413784,


#### 2. Explore Neighborhoods

In [206]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    LIMIT=20
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [207]:
toronto_venues = getNearbyVenues(names=df_toronto_filtered['Neighbourhood'],
                                   latitudes=df_toronto_filtered['Latitude'],
                                   longitudes=df_toronto_filtered['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [208]:
print(toronto_venues.shape)
toronto_venues.head()

(318, 7)


Neighborhood  Neighborhood Latitude  \
0                     The Beaches              43.676357   
1                     The Beaches              43.676357   
2                     The Beaches              43.676357   
3                     The Beaches              43.676357   
4  The Beaches West, India Bazaar              43.668999   

   Neighborhood Longitude                       Venue  Venue Latitude  \
0              -79.293031           Glen Manor Ravine       43.676821   
1              -79.293031           Glen Stewart Park       43.675278   
2              -79.293031         Glen Stewart Ravine       43.676300   
3              -79.293031      Balmy Beach Playground       43.676078   
4              -79.315572  British Style Fish & Chips       43.668723   

   Venue Longitude        Venue Category  
0       -79.293942                 Trail  
1       -79.294647                  Park  
2       -79.294784  Other Great Outdoors  
3       -79.290805            Playground  
4       -79.317139     Fish & Chips Shop

In [209]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                               20   
Business Reply Mail Processing Centre 969 Eastern                       2   
CN Tower, Bathurst Quay, Island airport, Harbou...                      1   
Cabbagetown, St. James Town                                            13   
Central Bay Street                                                     13   
Chinatown, Grange Park, Kensington Market                              20   
Church and Wellesley                                                   20   
Commerce Court, Victoria Hotel                                         20   
Davisville                                                             15   
Davisville North                                                        2   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                      5   
Design Exchange, Toronto Dominion Centre                               20   
Dovercourt Village, Dufferin                                            6   
First Canadian Place, Underground city                                 20   
Forest Hill North, Forest Hill West                                     1   
Harbord, University of Toronto                                          7   
Harbourfront East, Toronto Islands, Union Station                      20   
Harbourfront, Regent Park                                               7   
Lawrence Park                                                           2   
Little Portugal, Trinity                                               20   
Moore Park, Summerhill East                                             2   
Parkdale, Roncesvalles                                                  2   
Rosedale                                                                1   
Roselawn                                                                1   
Runnymede, Swansea                                                      2   
Ryerson, Garden District                                               20   
St. James Town                                                         15   
Stn A PO Boxes 25 The Esplanade                                        20   
Studio District                                                        15   
The Beaches                                                             4   
The Beaches West, India Bazaar                                          2   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                                20   
Business Reply Mail Processing Centre 969 Eastern                        2   
CN Tower, Bathurst Quay, Island airport, Harbou...                       1   
Cabbagetown, St. James Town                                             13   
Central Bay Street                                                      13   
Chinatown, Grange Park, Kensington Market                               20   
Church and Wellesley                                                    20   
Commerce Court, Victoria Hotel                                          20   
Davisville                                                              15   
Davisville North                                                         2   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                       5   
Design Exchange, Toronto Dominion Centre                                20   
Dovercourt Village, Dufferin                                             6   
First Canadian Place, Underground city                                  20   
Forest Hill North, Forest Hill West                                      1   
Harbord, University of Toronto                                           7   
Harbourfront East, Toronto Islands, Union Station                       20   
Harbourfront, Regent Park   

In [210]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 123 uniques categories.


#### c.3 Analyze Each Neighborhood

In [211]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Adult Boutique  American Restaurant  \
0            0                  0               0                    0   
1            0                  0               0                    0   
2            0                  0               0                    0   
3            0                  0               0                    0   
4            0                  0               0                    0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  Auto Workshop  \
0            0                    0                 0              0   
1            0                    0                 0              0   
2            0                    0                 0              0   
3            0                    0                 0              0   
4            0                    0                 0              0   

   BBQ Joint  Bagel Shop  Bakery  Bank  Bar  Beer Bar  Beer Store  \
0          0           0       0     0    0         0           0   
1          0           0       0     0    0         0           0   
2          0           0       0     0    0         0           0   
3          0           0       0     0    0         0           0   
4          0           0       0     0    0         0           0   

   Belgian Restaurant  Bookstore  Boutique  Breakfast Spot  Brewery  \
0                   0          0         0               0        0   
1                   0          0         0               0        0   
2                   0          0         0               0        0   
3                   0          0         0               0        0   
4                   0          0         0               0        0   

   Bubble Tea Shop  Burger Joint  Burrito Place  Café  Camera Store  \
0                0             0              0     0             0   
1                0             0              0     0             0   
2                0             0              0     0             0   
3                0             0              0     0             0   
4                0             0              0     0             0   

   Caribbean Restaurant  Cheese Shop  Church  Clothing Store  Cocktail Bar  \
0                     0            0       0               0             0   
1                     0            0       0               0             0   
2                     0            0       0               0             0   
3                     0            0       0               0             0   
4                     0            0       0               0             0   

   Coffee Shop  College Gym  Colombian Restaurant  Comic Shop  Concert Hall  \
0            0            0                     0           0             0   
1            0            0                     0           0             0   
2            0            0                     0           0             0   
3            0            0                     0           0             0   
4            0            0                     0           0             0   

   Convenience Store  Creperie  Cuban Restaurant  Deli / Bodega  Dessert Shop  \
0                  0         0                 0              0             0   
1                  0         0                 0              0             0   
2                  0         0                 0              0             0   
3                  0         0                 0              0             0   
4                  0         0                 0              0             0   

   Diner  Dog Run  Farmers Market  Fish & Chips Shop  Flower Shop  Food Court  \
0      0        0               0                  0            0           0   
1      0        0               0                  0            0           0   
2      0        0               0                  0            0           0   
3      0        0               0                  0            0           0   
4      0        0   

In [212]:
toronto_onehot.shape

(318, 123)

In [213]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                            Adelaide, King, Richmond         0.00   
1   Business Reply Mail Processing Centre 969 Eastern         0.00   
2   CN Tower, Bathurst Quay, Island airport, Harbo...         0.00   
3                         Cabbagetown, St. James Town         0.00   
4                                  Central Bay Street         0.00   
5           Chinatown, Grange Park, Kensington Market         0.00   
6                                Church and Wellesley         0.00   
7                      Commerce Court, Victoria Hotel         0.00   
8                                          Davisville         0.00   
9                                    Davisville North         0.00   
10  Deer Park, Forest Hill SE, Rathnelly, South Hi...         0.00   
11           Design Exchange, Toronto Dominion Centre         0.00   
12                       Dovercourt Village, Dufferin         0.00   
13             First Canadian Place, Underground city         0.00   
14                Forest Hill North, Forest Hill West         0.00   
15                     Harbord, University of Toronto         0.00   
16  Harbourfront East, Toronto Islands, Union Station         0.00   
17                          Harbourfront, Regent Park         0.00   
18                                      Lawrence Park         0.00   
19                           Little Portugal, Trinity         0.05   
20                        Moore Park, Summerhill East         0.00   
21                             Parkdale, Roncesvalles         0.00   
22                                           Rosedale         0.00   
23                                           Roselawn         0.00   
24                                 Runnymede, Swansea         0.00   
25                           Ryerson, Garden District         0.00   
26                                     St. James Town         0.00   
27                    Stn A PO Boxes 25 The Esplanade         0.00   
28                                    Studio District         0.00   
29                                        The Beaches         0.00   
30                     The Beaches West, India Bazaar         0.00   

    Accessories Store  Adult Boutique  American Restaurant  Art Gallery  \
0                 0.0            0.00                 0.00         0.00   
1                 0.0            0.00                 0.00         0.00   
2                 0.0            0.00                 0.00         0.00   
3                 0.0            0.00                 0.00         0.00   
4                 0.0            0.00                 0.00         0.00   
5                 0.0            0.00                 0.00         0.00   
6                 0.0            0.05                 0.00         0.00   
7                 0.0            0.00                 0.00         0.05   
8                 0.0            0.00                 0.00         0.00   
9                 0.0            0.00                 0.00         0.00   
10                0.0            0.00                 0.00         0.00   
11                0.0            0.00                 0.00         0.05   
12                0.0            0.00                 0.00         0.00   
13                0.0            0.00                 0.05         0.05   
14                1.0            0.00                 0.00         0.00   
15                0.0            0.00                 0.00         0.00   
16                0.0            0.00                 0.00         0.00   
17                0.0            0.00                 0.00         0.00   
18                0.0            0.00                 0.00         0.00   
19                0.0            0.00                 0.00         0.00   
20                0.0            0.00                 0.00         0.00   
21                0.0            0.00                 0.00         0.00   
22                0.0            0.00                 0.00         0.00   


In [214]:
toronto_grouped.shape

(31, 123)

In [215]:
#Lets princ each neighborhood with the top5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0        Steakhouse  0.15
1  Asian Restaurant  0.10
2       Coffee Shop  0.10
3             Hotel  0.05
4      Noodle House  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0       Auto Workshop   0.5
1             Brewery   0.5
2         Yoga Studio   0.0
3  Mexican Restaurant   0.0
4                Park   0.0


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                   venue  freq
0  Performing Arts Venue   1.0
1            Yoga Studio   0.0
2      Outdoor Sculpture   0.0
3   Other Great Outdoors   0.0
4        Organic Grocery   0.0


----Cabbagetown, St. James Town----
                   venue  freq
0                   Café  0.15
1                 Bakery  0.08
2  General Entertainment  0.08
3            Pizza Place  0.08
4      Outdoor Sculpture  0.08


----Central Bay Street----
                       venu

In [216]:
#functions fo short venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [217]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood  1st Most Common Venue  \
0                           Adelaide, King, Richmond             Steakhouse   
1  Business Reply Mail Processing Centre 969 Eastern          Auto Workshop   
2  CN Tower, Bathurst Quay, Island airport, Harbo...  Performing Arts Venue   
3                        Cabbagetown, St. James Town                   Café   
4                                 Central Bay Street            Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0           Coffee Shop      Asian Restaurant            Food Court   
1               Brewery              Wine Bar            Food Court   
2          Cocktail Bar           College Gym  Colombian Restaurant   
3     Outdoor Sculpture                Bakery           Pizza Place   
4        Sandwich Place         Smoothie Shop              Pharmacy   

  5th Most Common Venue 6th Most Common Venue      7th Most Common Venue  \
0           Opera House  Colombian Restaurant           Greek Restaurant   
1            Comic Shop          Concert Hall          Convenience Store   
2            Comic Shop          Concert Hall          Convenience Store   
3                 Diner                Market                 Restaurant   
4                  Café             Bookstore  Middle Eastern Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0          Concert Hall        Breakfast Spot                    Bar  
1              Creperie      Cuban Restaurant          Deli / Bodega  
2              Creperie      Cuban Restaurant          Deli / Bodega  
3            Beer Store    Italian Restaurant      Indian Restaurant  
4    Italian Restaurant         Deli / Bodega                Dog Run

#### c.4 Cluster Neighborhoods

In [218]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0])

In [219]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

Postcode          Borough                   Neighbourhood   Latitude  \
0      M4E     East Toronto                     The Beaches  43.676357   
1      M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2      M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
3      M4M     East Toronto                 Studio District  43.659526   
4      M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031             0.0                  Park                 Trail   
1 -79.352188             NaN                   NaN                   NaN   
2 -79.315572             3.0                  Park     Fish & Chips Shop   
3 -79.340923             0.0           Coffee Shop                  Café   
4 -79.388790             0.0  Gym / Fitness Center                  Lake   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0            Playground  Other Great Outdoors           College Gym   
1                   NaN                   NaN                   NaN   
2              Wine Bar           College Gym  Colombian Restaurant   
3        Clothing Store                Bakery     Convenience Store   
4           Flower Shop           College Gym  Colombian Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0  Colombian Restaurant           Coffee Shop            Comic Shop   
1                   NaN                   NaN                   NaN   
2            Comic Shop          Concert Hall     Convenience Store   
3           Cheese Shop             Bookstore        Sandwich Place   
4            Comic Shop          Concert Hall     Convenience Store   

  9th Most Common Venue 10th Most Common Venue  
0        Farmers Market      Convenience Store  
1                   NaN                    NaN  
2              Creperie       Cuban Restaurant  
3                   Bar                   Bank  
4              Creperie       Cuban Restaurant

In [226]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[np.int(cluster)-1],
        fill=True,
        fill_color=rainbow[np.int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: cannot convert float NaN to integer

In [221]:
#### c.5 Examine Clusters
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0       East Toronto             0.0                  Park   
3       East Toronto             0.0           Coffee Shop   
4    Central Toronto             0.0  Gym / Fitness Center   
5    Central Toronto             0.0     Convenience Store   
7    Central Toronto             0.0          Dessert Shop   
9    Central Toronto             0.0                  Park   
11  Downtown Toronto             0.0                  Café   
12  Downtown Toronto             0.0          Burger Joint   
13  Downtown Toronto             0.0           Coffee Shop   
14  Downtown Toronto             0.0           Coffee Shop   
15  Downtown Toronto             0.0           Coffee Shop   
17  Downtown Toronto             0.0           Coffee Shop   
18  Downtown Toronto             0.0            Steakhouse   
19  Downtown Toronto             0.0           Coffee Shop   
20  Downtown Toronto             0.0           Coffee Shop   
21  Downtown Toronto             0.0           Coffee Shop   
25  Downtown Toronto             0.0           Coffee Shop   
26  Downtown Toronto             0.0                  Café   
28  Downtown Toronto             0.0   Moroccan Restaurant   
29  Downtown Toronto             0.0             Gastropub   
31      West Toronto             0.0  Gym / Fitness Center   
32      West Toronto             0.0                   Bar   
35      West Toronto             0.0                Garden   
36      West Toronto             0.0            Hobby Shop   
37      East Toronto             0.0         Auto Workshop   

      2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                     Trail            Playground  Other Great Outdoors   
3                      Café        Clothing Store                Bakery   
4                      Lake           Flower Shop           College Gym   
5            Breakfast Spot              Wine Bar           Flower Shop   
7               Coffee Shop    Italian Restaurant                  Café   
9               Supermarket          Liquor Store    Light Rail Station   
11        Outdoor Sculpture                Bakery           Pizza Place   
12        Martial Arts Dojo       Bubble Tea Shop              Boutique   
13           History Museum  Gym / Fitness Center        Breakfast Spot   
14       Mexican Restaurant                 Plaza         Movie Theater   
15      Japanese Restaurant             Gastropub           Pizza Place   
17           Sandwich Place         Smoothie Shop              Pharmacy   
18              Coffee Shop      Asian Restaurant            Food Court   
19  New American Restaurant       Bubble Tea Shop                 Plaza   
20               Restaurant         Deli / Bodega                   Gym   
21                     Café            Restaurant         Deli / Bodega   
25              Pizza Place             BBQ Joint        Sandwich Place   
26    Vietnamese Restaurant    Mexican Restaurant  Caribbean Restaurant   
28             Concert Hall            Steakhouse        Breakfast Spot   
29              Coffee Shop                  Café            Restaurant   
31              Supermarket              Pharmacy           Music Venue   
32                 Wine Bar     French Restaurant           Pizza Place   
35       Light Rail Station              Wine Bar           Flower Shop   
36         Sushi Restaurant              Wine Bar           Flower Shop   
37                  Brewery              Wine Bar            Food Court   

   5th Most Common Venue   6th Most Common Venue      7th Most Common Venue  \
0            College Gym    Colombian Restaurant                Coffee Shop   
3      Convenience Store             Cheese Shop                  Bookstore   
4   Colombian Restaurant              Comic Shop               Concert Hall   
5   Colombian Restaurant              Comic Shop               Concert Hall   
7     Seafood Restaurant             Pizza Place      

In [222]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
23  Central Toronto             1.0     Accessories Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23              Wine Bar            Food Court  Colombian Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23            Comic Shop          Concert Hall     Convenience Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
23              Creperie      Cuban Restaurant          Deli / Bodega

In [223]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels  1st Most Common Venue  \
27  Downtown Toronto             2.0  Performing Arts Venue   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
27          Cocktail Bar           College Gym  Colombian Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
27            Comic Shop          Concert Hall     Convenience Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
27              Creperie      Cuban Restaurant          Deli / Bodega

In [224]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
2       East Toronto             3.0                  Park   
8    Central Toronto             3.0                  Park   
10  Downtown Toronto             3.0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
2      Fish & Chips Shop              Wine Bar           College Gym   
8          Moving Target     Fish & Chips Shop           College Gym   
10              Wine Bar           College Gym  Colombian Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
2   Colombian Restaurant            Comic Shop          Concert Hall   
8   Colombian Restaurant            Comic Shop          Concert Hall   
10            Comic Shop          Concert Hall     Convenience Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
2      Convenience Store              Creperie       Cuban Restaurant  
8      Convenience Store              Creperie       Cuban Restaurant  
10              Creperie      Cuban Restaurant          Deli / Bodega

In [225]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels    1st Most Common Venue  \
22  Central Toronto             4.0  Health & Beauty Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
22           Coffee Shop  Colombian Restaurant            Comic Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
22          Concert Hall     Convenience Store              Creperie   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
22      Cuban Restaurant         Deli / Bodega           Dessert Shop